<h1 style="color:gold; background-color:black; padding:20px; font-size:40px; text-align:center">TRADUCTOR ESPAÑOL - QUECHUA</h1>

<h1>Importar librerias a utilizar</h1>

In [1]:
import json
import numpy as np
import os
import pandas as pd

In [2]:
import pydot
import graphviz

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

<h1>Definir funciones auxiliares</h1>

<h3 style="color:crimson">Imprime barra de progreso</h3>

In [4]:
def imprimir_barra_progreso_v2(progreso, total):
    # https://www.youtube.com/watch?v=x1eaT88vJUA
    
    p = int(100 * (progreso + 1)/total)

    # Alt+291: █
    barra = '█'*p + '-'*(100-p) 

    print("\r|{}| {}%".format(barra, p), end='\r')

<h1>Importar datos</h1>

<p>Los archivos contienen palabras/oraciones paralelas
<br>
La primera columna está en español y la segunda en quechua</p>

In [5]:
# Crear un dataframe vació que almacenará las traducciones (translations)
df_trans = pd.DataFrame(columns=["español","quechua"])
df_trans

,español,quechua


<p>Se utilizó un tab ("\t") como separador de columnas para evitar confusiones con las comas propias de las oraciones</p>

<h3 style="color:crimson">Importa DataFrames</h3>

In [6]:
arr_opciones = [[40,[40,50,60]],
               [50,[50,63,75]],
               [60,[60,75,90]]]

In [7]:
print("Longitudes disponibles en español")
for i in range(3):
    print("{}) - {}".format(i, arr_opciones[i][0]) )
MAX_ES_index = int(input("Ingrese la longitud máxima en español: "))

print("\n" + "-"*100)
print("Longitudes disponibles en quechua")
for j in range(3):
    print("{}) - {}".format(j, arr_opciones[MAX_ES_index][1][j]))
MAX_QU_index = int(input("Ingrese la longitud máxima en quechua: "))

Longitudes disponibles en español
0) - 40
1) - 50
2) - 60
Ingrese la longitud máxima en español: 0

----------------------------------------------------------------------------------------------------
Longitudes disponibles en quechua
0) - 40
1) - 50
2) - 60
Ingrese la longitud máxima en quechua: 0


In [8]:
MAX_ES = arr_opciones[MAX_ES_index][0]
MAX_QU = arr_opciones[MAX_ES_index][1][MAX_QU_index]

folder_consolidado = "./Datos/consolidado_{}_{}/".format(MAX_ES, MAX_QU)
print("obteniendo datos de {}".format(folder_consolidado))

sub_paths = [folder_consolidado,
             #"./Datos/palabras/", "./Datos/grupos/",
             #"./Datos/libros/", "./Datos/oraciones/"
            ]

for sub_path in sub_paths:
    arr_category = os.listdir(sub_path)

    for item in arr_category:
        if item[-4:] == ".csv":
            file = "{}{}".format(sub_path,item)
            df_temp = pd.read_csv(file, encoding="utf-8", sep="\t")
            
            df_trans = pd.concat([df_trans, df_temp]
                                 , ignore_index = True)

obteniendo datos de ./Datos/consolidado_40_40/


In [9]:
df_trans.head(100)

,español,quechua
0,Ve.,Riy.
1,Vete.,Lluqsiy.
2,Vaya.,Waw.
3,Váyase.,Lluqsiy.
4,Hola.,Allinllachu.
...,...,...
95,Me escapé.,Ayqirqanim.
96,Huía.,ayqirqa
97,Me escapaba.,Ayqichkarqanim.
98,Yo lo sé.,Chaytaqa yachani.


In [10]:
df_trans.tail(100)

,español,quechua
90213,Como un padre decrépito disfruta,Imaynan huk decrepit tayta kusikun
90214,"al ver de su hijo las empresas jóvenes,","churinpa wayna empresankunata rikuspa,"
90215,"así yo, mutilado por la suene,","chaymi ñoqa, t’oqyaywan mutilado, ."
90216,"el poder o el ingenio, uno o todos,","atiy utaq ingenio, huk utaq llapan, ."
90217,"para colmarme a mí con su opulencia,","qhapaq kayninwan hunt'achiwananpaq,"
...,...,...
90308,"Pobre alma, centro de culpable limo","Wakcha alma, huchayuq limo centro"
90309,"a la que burla, indócil, quien la ciñe,","burlakuq, mana kamachikuq, chumpikuq, ."
90310,ornando tu morada pasajera?,temporal tiyasqaykita sumaqchaspa?
90311,"Vive, alma, a expensas de tu servidor;","Kawsay, alma, kamachiykiq qolqenwan;"


<h3 style="color:crimson">Eliminar frases muy largas</h3>
<p>Los tres arreglos de numpy deben entrar en los 8 GB de VRAM, y dejar espacio para el resto de variables</p>

In [11]:
print("Longitud inicial del arreglo {}".format(len(df_trans)))

Longitud inicial del arreglo 90313


In [12]:
for index, registro in df_trans.iterrows():
    txt_es = registro[0].strip()
    txt_qu = registro[1].strip()
    
    if (len(txt_es) > MAX_ES) or (len(txt_qu) > MAX_QU):
        #print("registro eliminado")
        df_trans = df_trans.drop(index)
        
    if index % 5000 == 0:
        print("Rggistro {} analizado".format(index))

Rggistro 0 analizado
Rggistro 5000 analizado
Rggistro 10000 analizado
Rggistro 15000 analizado
Rggistro 20000 analizado
Rggistro 25000 analizado
Rggistro 30000 analizado
Rggistro 35000 analizado
Rggistro 40000 analizado
Rggistro 45000 analizado
Rggistro 50000 analizado
Rggistro 55000 analizado
Rggistro 60000 analizado
Rggistro 65000 analizado
Rggistro 70000 analizado
Rggistro 75000 analizado
Rggistro 80000 analizado
Rggistro 85000 analizado
Rggistro 90000 analizado


In [13]:
print("Nueva longitud del arreglo {}".format(len(df_trans)))

Nueva longitud del arreglo 90313


<h1>Configuración</h1>

In [14]:
batch_size = 128 # tamño de los lotes para entrenamiento
epochs = 250 # Número de epochs
latent_dim = 256 # dimensión del espacio latente para el encoder
num_samples = len(df_trans)

<h1>Preparar los datos</h1>

In [15]:
# Vectoriza los datos
i=0
target_text= ''

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [16]:
for index, registro in df_trans.iterrows():    
    
    input_text = registro[0].strip()
    target_text = registro[1].strip()
    
    
    if (index<100):
        pad_dashes = " " + "-"*(20-len(input_text)) + " "
        print("{} \t I: |{}| {} T: |{}|"
              .format(index, input_text, pad_dashes, target_text))        

    
    # Usaremos "tab" como el  caracter de inicio (start sequence)
    # para los targets, y "\n" como el caracter de fin de secuencia "end sequence"
    target_text = "\t" + target_text + "\n"
    # sube las líneas a  las listas
    input_texts.append(input_text)
    target_texts.append(target_text)
  
    # completa los conjuntos de caracteres si es necesario
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
    i=i+1

0 	 I: |Ve.|  -----------------  T: |Riy.|
1 	 I: |Vete.|  ---------------  T: |Lluqsiy.|
2 	 I: |Vaya.|  ---------------  T: |Waw.|
3 	 I: |Váyase.|  -------------  T: |Lluqsiy.|
4 	 I: |Hola.|  ---------------  T: |Allinllachu.|
5 	 I: |¡Corre!|  -------------  T: |¡Runa!|
6 	 I: |¡Corran!|  ------------  T: |Paway!|
7 	 I: |¡Huye!|  --------------  T: |¡Ascapa!|
8 	 I: |¡Corra!|  -------------  T: |Paway!|
9 	 I: |¡Corred!|  ------------  T: |Paway!|
10 	 I: |Corra.|  --------------  T: |Paway.|
11 	 I: |Corred.|  -------------  T: |Paway.|
12 	 I: |¿Quién?|  -------------  T: |Pi?|
13 	 I: |¡Órale!|  -------------  T: |Waw!|
14 	 I: |¡Inclínate!|  ---------  T: |Kumuy!|
15 	 I: |¡Fuego!|  -------------  T: |Nina!|
16 	 I: |¡Incendio!|  ----------  T: |Nina!|
17 	 I: |¡Disparad!|  ----------  T: |disparar!|
18 	 I: |¡Ayuda!|  -------------  T: |Yanapay!|
19 	 I: |¡Socorro! ¡Auxilio!|  -  T: |Yanapay! Yanapay!|
20 	 I: |¡Auxilio!|  -----------  T: |Yanapay!|
21 	 I: |Escóndete.|  ---

In [17]:
# Convierte los dos conjuntos de caracteres
# en dos listas ordenadas
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))  
# calcule el número de tokens (caracteres) en ambos lados
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
# calcula la máxima longitud de las secuencias en cada lado
max_encoder_seq_length = max([len(text) for text in input_texts])
max_decoder_seq_length = max([len(text) for text in target_texts])

In [18]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
print("preparando datos...")

Number of samples: 90313
Number of unique input tokens: 110
Number of unique output tokens: 104
Max sequence length for inputs: 40
Max sequence length for outputs: 42
preparando datos...


In [19]:
# crea diccionarios de tokens
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# crea los tensores 1-hot para el encoder y el decoder
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

In [20]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # Modificar los valores de la matriz del encoder
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    
    
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    
    
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

print ("\n....")
print("datos preparados")


....
datos preparados


<h1>Construir el modelo</h1>

<h3 style="color:crimson">Encoder</h3>

In [21]:
# define una secuencia de entrada y la procesa
encoder_inputs = Input(shape = (None, num_encoder_tokens))

# capa recurrente del encoder
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# Descartamos las salidas (encoder_outputs)
# solamente se conserva las memoria de  corto (state_h) y 
# largo plazo(state_c)
encoder_states = [state_h, state_c]

<h3 style="color:crimson">Decoder</h3>

In [22]:
# Configuramos el decoder, usando 'encoder_states' como estado inicial
decoder_inputs = Input(shape= (None, num_decoder_tokens))

# capa recurrente del decoder
# Configuramos nuestro decodificador para devolver secuencias de salida completas,
# y también para devolver estados internos. No usamos los
# estados retornados en el modelo de entrenamiento, pero los usaremos en inferencia.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _,_ = decoder_lstm(decoder_inputs,initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

<h3 style="color:crimson">Modelo Completo</h3>

In [23]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [24]:
model.summary()
plot_model(model, to_file='../Imagenes/s2s.png', 
           show_shapes=True)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 110)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 104)]  0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        375808      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

<h1>Entrenar el modelo</h1>

In [25]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [26]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/250
565/565 [==============================] - 19s 27ms/step - loss: 1.5433 - accuracy: 0.5709 - val_loss: 1.4891 - val_accuracy: 0.5748
Epoch 2/250
565/565 [==============================] - 14s 25ms/step - loss: 1.0688 - accuracy: 0.6778 - val_loss: 1.3151 - val_accuracy: 0.6207
Epoch 3/250
565/565 [==============================] - 14s 25ms/step - loss: 0.9299 - accuracy: 0.7178 - val_loss: 1.2146 - val_accuracy: 0.6515
Epoch 4/250
565/565 [==============================] - 14s 25ms/step - loss: 0.8445 - accuracy: 0.7444 - val_loss: 1.1378 - val_accuracy: 0.6747
Epoch 5/250
565/565 [==============================] - 14s 25ms/step - loss: 0.7839 - accuracy: 0.7631 - val_loss: 1.0901 - val_accuracy: 0.6913
Epoch 6/250
565/565 [==============================] - 14s 25ms/step - loss: 0.7329 - accuracy: 0.7786 - val_loss: 1.0404 - val_accuracy: 0.7049
Epoch 7/250
565/565 [==============================] - 14s 25ms/step - loss: 0.6897 - accuracy: 0.7917 - val_loss: 1.0044 - val_ac

565/565 [==============================] - 14s 25ms/step - loss: 0.2628 - accuracy: 0.9194 - val_loss: 0.9919 - val_accuracy: 0.7714
Epoch 58/250
565/565 [==============================] - 14s 25ms/step - loss: 0.2600 - accuracy: 0.9202 - val_loss: 0.9986 - val_accuracy: 0.7716
Epoch 59/250
565/565 [==============================] - 14s 25ms/step - loss: 0.2576 - accuracy: 0.9209 - val_loss: 1.0000 - val_accuracy: 0.7707
Epoch 60/250
565/565 [==============================] - 14s 25ms/step - loss: 0.2553 - accuracy: 0.9219 - val_loss: 1.0065 - val_accuracy: 0.7705
Epoch 61/250
565/565 [==============================] - 14s 25ms/step - loss: 0.2531 - accuracy: 0.9223 - val_loss: 1.0152 - val_accuracy: 0.7699
Epoch 62/250
565/565 [==============================] - 14s 25ms/step - loss: 0.2509 - accuracy: 0.9231 - val_loss: 1.0173 - val_accuracy: 0.7701
Epoch 63/250
565/565 [==============================] - 14s 25ms/step - loss: 0.2485 - accuracy: 0.9237 - val_loss: 1.0314 - val_accuracy

565/565 [==============================] - 14s 25ms/step - loss: 0.1853 - accuracy: 0.9423 - val_loss: 1.2759 - val_accuracy: 0.7614
Epoch 114/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1854 - accuracy: 0.9421 - val_loss: 1.2836 - val_accuracy: 0.7603
Epoch 115/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1846 - accuracy: 0.9424 - val_loss: 1.2839 - val_accuracy: 0.7613
Epoch 116/250
565/565 [==============================] - 14s 24ms/step - loss: 0.1843 - accuracy: 0.9423 - val_loss: 1.2848 - val_accuracy: 0.7602
Epoch 117/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1834 - accuracy: 0.9426 - val_loss: 1.2914 - val_accuracy: 0.7599
Epoch 118/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1818 - accuracy: 0.9434 - val_loss: 1.3039 - val_accuracy: 0.7601
Epoch 119/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1818 - accuracy: 0.9432 - val_loss: 1.3060 - val_ac

Epoch 169/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1583 - accuracy: 0.9498 - val_loss: 1.4550 - val_accuracy: 0.7572
Epoch 170/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1585 - accuracy: 0.9497 - val_loss: 1.4605 - val_accuracy: 0.7567
Epoch 171/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1590 - accuracy: 0.9495 - val_loss: 1.4556 - val_accuracy: 0.7569
Epoch 172/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1584 - accuracy: 0.9498 - val_loss: 1.4540 - val_accuracy: 0.7574
Epoch 173/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1566 - accuracy: 0.9503 - val_loss: 1.4631 - val_accuracy: 0.7564
Epoch 174/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1565 - accuracy: 0.9504 - val_loss: 1.4752 - val_accuracy: 0.7561
Epoch 175/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1572 - accuracy: 0.9500 - val_loss: 1

Epoch 225/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1438 - accuracy: 0.9540 - val_loss: 1.5730 - val_accuracy: 0.7544
Epoch 226/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1446 - accuracy: 0.9537 - val_loss: 1.5739 - val_accuracy: 0.7552
Epoch 227/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1433 - accuracy: 0.9541 - val_loss: 1.5800 - val_accuracy: 0.7551
Epoch 228/250
565/565 [==============================] - 14s 24ms/step - loss: 0.1434 - accuracy: 0.9539 - val_loss: 1.5799 - val_accuracy: 0.7541
Epoch 229/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1446 - accuracy: 0.9534 - val_loss: 1.5765 - val_accuracy: 0.7543
Epoch 230/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1437 - accuracy: 0.9538 - val_loss: 1.5842 - val_accuracy: 0.7537
Epoch 231/250
565/565 [==============================] - 14s 25ms/step - loss: 0.1434 - accuracy: 0.9540 - val_loss: 1

<h1>Guardar el modelo</h1>

<h3 style="color:crimson">Diccionarios</h3>

In [27]:
with open("./Modelos/s2q/input_token_index.txt", "w") as f:
    json.dump(input_token_index, f)

In [28]:
with open("./Modelos/s2q/target_token_index.txt", "w") as f:
    json.dump(target_token_index, f)

In [29]:
with open("./Modelos/s2q/history.txt", "w") as f:
    json.dump(history.history, f)

<h3 style="color:crimson">Valores puntuales</h3>

In [30]:
print("num_decoder_tokens: {}".format(num_decoder_tokens))
print("max_decoder_seq_length: {}".format(max_decoder_seq_length))

print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

num_decoder_tokens: 104
max_decoder_seq_length: 42
Number of unique input tokens: 110
Number of unique output tokens: 104
Max sequence length for inputs: 40
Max sequence length for outputs: 42


In [32]:
with open("./Modelos/s2q/otros.txt", "w") as f:
    f.write("num_encoder_tokens = {}\n".format(num_encoder_tokens))
    f.write("num_decoder_tokens = {}\n".format(num_decoder_tokens))
    
    f.write("max_encoder_seq_length = {}\n".format(max_encoder_seq_length))
    f.write("max_decoder_seq_length = {}".format(max_decoder_seq_length))

<h3 style="color:crimson">Arreglo numpy</h3>
<p>Sólo las primeras 100 posiciones</p>

In [33]:
file_name = "./Modelos/s2q/encoder_input_data_sample.npy"
with open(file_name, "wb") as f:
    np.save(f, encoder_input_data[:100])

<h3 style="color:crimson">Modelo</h3>

In [34]:
# s2q = Spanish to Quechua
model.save("./Modelos/s2q/spanish_to_quechua")

INFO:tensorflow:Assets written to: ./Modelos/s2q/spanish_to_quechua\assets


INFO:tensorflow:Assets written to: ./Modelos/s2q/spanish_to_quechua\assets


In [35]:
model.save("./Modelos/s2q/spanish_to_quechua_file.h5")